In [ ]:
from db_import import *
from datetime import datetime, timedelta
import numpy as np
import warnings
warnings.filterwarnings("ignore")
pd.set_option('display.max_columns', 30)
pd.options.display.float_format = '{:.2f}'.format
from forex_python.converter import CurrencyRates

# Data Import
end_date = datetime.now().strftime('%Y-%m-%d')
start_date = (datetime.now()-timedelta(days=7)).strftime('%Y-%m-%d')
query_obj = Queries(start_date, end_date)
db_obj = DBImport(db_type='cscart')

recommender = db_obj.data_import(query_obj.recommender_query)
user = db_obj.data_import(query_obj.user_query)
analytics = db_obj.data_import(query_obj.analytics_query)

col_name_dict = {
    'recommender_user_id': '추천인아이디',
    'reco_count': '추천수',
    'reco_order_count': '추천주문회수',
    'reco_order_rate': '추천주문전환율(%)',
    'reco_sale_amt': '추천주문매출액(원)',
    'recommend_user_list': '추천자리스트',
    'user_name': '추천인'
}

# 유저별 추천 받은 수
user_count = recommender.groupby('recommender_user_id')['user_id'].nunique().reset_index()
user_count = user_count.rename(columns={'user_id': 'reco_count'})
# 유저별 추천 주문 수
order_count = recommender[recommender['order_id']!=0].groupby('recommender_user_id')['order_id'].nunique().reset_index()
order_count = order_count.rename(columns={'order_id': 'reco_order_count'})
# 유저별 추천주문 금액
exchange_rate = CurrencyRates() # 환율 고려하여 원단위 매출 연산
usd_krw = exchange_rate.get_rates('USD')['KRW']
analytics['krw_price'] = analytics.apply(lambda x: x['marked_down_price'] * usd_krw if x['currency']=='USD' else x['marked_down_price'], axis=1)
analytics['krw_sales'] = analytics['krw_price'] * analytics['product_qty']
order_sale = analytics.groupby('order_id')['krw_sales'].sum().reset_index()
reco_sale = pd.merge(recommender, order_sale, on='order_id', how='left')
reco_sale = reco_sale.groupby('recommender_user_id')['krw_sales'].sum().reset_index()
reco_sale = reco_sale.rename(columns={'krw_sales': 'reco_sale_amt'})
# 유저별 추천인 목록
reco_list = recommender.groupby('recommender_user_id')['user_id'].apply(list).reset_index()
reco_list = reco_list.rename(columns={'user_id': 'recommend_user_list'})
# 추천인명
user['user_name'] = user['lastname'] + user['firstname']
user_name = user.groupby('user_id')['user_name'].sum().reset_index()
user_name = user_name.rename(columns={'user_id': 'recommender_user_id'})

# 종합
final_result = pd.merge(user_count, order_count, on='recommender_user_id', how='left')
final_result['reco_order_rate'] = np.round(final_result['reco_order_count'] / final_result['reco_count'], 4)*100
final_result = pd.merge(final_result, reco_sale, on='recommender_user_id', how='left').fillna(0)
final_result = pd.merge(final_result, reco_list, on='recommender_user_id', how='left')
final_result = pd.merge(final_result, user_name, on='recommender_user_id', how='left')
final_result = final_result.sort_values(by='reco_count', ascending=False)
final_result = final_result[[
    'recommender_user_id', 'user_name', 'reco_count', 'reco_order_count',
    'reco_order_rate', 'reco_sale_amt', 'recommend_user_list'
]] # 컬럼 순서 정리
final_result = final_result.rename(columns=col_name_dict)

In [ ]:
final_result